In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
import torch

In [ ]:
assert torch.cuda.is_available()

In [ ]:
from PIL import Image
from PIL.Image import Dither

In [ ]:
import numpy as np

In [ ]:
import art_utils

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import io

## Login

- Create a login
- Create a token (fineGrained)
- Edit token access and add "Read access to contents of all public gated repos you can access"


In [ ]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [ ]:
width = 960
height = 680
my_dpi = 96

fig, ax = plt.subplots(
    1,1, figsize=(width/my_dpi, height/my_dpi), dpi=my_dpi
)

c1='#ffff'
c2='#0000'
n=500

for x in range(n+1):
    ax.axvline(x, color=colorFader(c1,c2,x/n), linewidth=4) 
ax.axis('off')

buf = io.BytesIO()
fig.savefig(buf, dpi=my_dpi)
buf.seek(0)
image = Image.open(buf)

## Load pipe

In [ ]:
from diffusers import StableDiffusion3Pipeline

In [ ]:
pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16)

In [ ]:
pipe = pipe.to("cuda")

## Selecting a bad prompt
Select a prompt that will generate a picture with lots of "grey" area

In [ ]:
prompt = "by Kazuo Umezu, horror, detailed, madness fine lines, surreal, color depth, white background, landscape"
prompt = "greay paint gradient, from white to black, colourful, high contrast"
prompt = "painting of a colorfull landscape"
#prompt = "comic artwork of two beautiful women atop epic swiss mountain with snow with the words 'happy birthday' in the snow, bw"

In [ ]:
meta = pipe(
    prompt,
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=7.0,
    num_images_per_prompt=1,
    width=960,
    height=688
)

images = meta.images
image = images[0]

In [ ]:
image = image.resize((960, 680))

In [ ]:
meta

In [ ]:
image

## Try dithering and send to 

In [ ]:
# Default dithering for PIL is Floyd-steinberg
image_f = image.convert("1", dither=Dither.FLOYDSTEINBERG)
image_f

In [ ]:
image_a = art_utils.atkinson_dither(image)

In [ ]:
image_a

In [ ]:
GRAY1  = 0xff #white
GRAY2  = 0xC0
GRAY3  = 0x80 #gray
GRAY4  = 0x00 #Blackest
palette = np.array([GRAY1, GRAY2, GRAY3, GRAY4])
palette = np.array([GRAY1, GRAY4])
print(palette)

In [ ]:
image_g = art_utils.atkinson_dither_with_palette(image, palette)

In [ ]:
image_g

## Compare in one photo

In [ ]:
# Get dimensions
width, height = image.size

# Find center
center_x = int(width/2)
center_y = int(height/2)

# Split two images
a = image_a.crop((0, 0, center_x, height))
b = image_g.crop((center_x, 0, width, height))

# Merge into one
image_m = image.copy()
image_m.paste(a, (0,0))
image_m.paste(b, (center_x,0))

In [ ]:
image_m

# Test by sending to photo

In [ ]:
from art_utils import network_utils

In [ ]:
r = network_utils.send_photo(image_g, "http://192.168.1.26:8080/display/image?useGrey=True")
r